Install packages/dependancies

In [ ]:
!pip install pytesseract transformers datasets evaluate rouge-score nltk tensorboard py7zr --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 40.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 50.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.3/412.3 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.9/138.9 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 9.4 MB/s

In [ ]:
import torch
import json
from tqdm import tqdm
import torch.nn as nn
from torch.optim import Adam
import nltk
import spacy
import string
import evaluate  # Bleu
from torch.utils.data import Dataset, DataLoader, RandomSampler
import pandas as pd
import numpy as np
import transformers
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from transformers import T5Tokenizer, T5Model, T5ForConditionalGeneration, T5TokenizerFast

import warnings
warnings.filterwarnings("ignore")

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
model_id="google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Verify if the summarization task works. It is observed that model is Performing better with foundational training.

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-small")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-small")

input_text = "summarize the text: For executive leaders, protecting the organization against risk is mission number one, and there is a long list of risks associated with generative AI: hallucinations, factual inaccuracies, data leakage, privacy concerns, security breaches, lack of attribution, copyright violations, and biased information. Leaders want to protect their data, IP, people, and business models, leading many to pause plans and put safeguards in place"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids

outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0]))

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


<pad>The generative AI is a key tool in the organization's strategy for protecting its data


Verify if English to French translation task works. It is observed that model is Performing better with foundational training.

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-small")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-small")

input_text = "translate English to French: How old are you?"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids

outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0]))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


<pad> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk>


Verify if Q&A task works. It is observed that model is Not performing better with foundational training.

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")

def query_from_list(query, options):
    t5query = f"""Question: Select the item from this list which is "{query}". Context: * {" * ".join(options)}"""
    inputs = tokenizer(t5query, return_tensors="pt")
    outputs = model.generate(**inputs, max_new_tokens=20)
    return tokenizer.batch_decode(outputs, skip_special_tokens=True)

tests = ["the first one", "the fish", "the chicken", "2nd", "bbq", "salmon", "roasted turkey", "dried halibut"]
options = ["Barbecue Chicken", "Smoked Salmon"]
for t in tests:
    result = query_from_list(t, options)
    print(f"{t:<24} {result[0]}")

the first one            Barbecue Chicken
the fish                 Barbecue Chicken
the chicken              Barbecue Chicken
2nd                      Barbecue Chicken
bbq                      Barbecue Chicken
salmon                   Barbecue Chicken
roasted turkey           Barbecue Chicken
dried halibut            Barbecue Chicken


However with flan-t5-base, Q&A tasks Performing better with foundational training.

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")

def query_from_list(query, options):
    t5query = f"""Question: Select the item from this list which is "{query}". Context: * {" * ".join(options)}"""
    inputs = tokenizer(t5query, return_tensors="pt")
    outputs = model.generate(**inputs, max_new_tokens=20)
    return tokenizer.batch_decode(outputs, skip_special_tokens=True)

tests = ["the first one", "the fish", "the chicken", "2nd", "bbq", "salmon", "roasted turkey", "dried halibut"]
options = ["Barbecue Chicken", "Smoked Salmon"]
for t in tests:
    result = query_from_list(t, options)
    print(f"{t:<24} {result[0]}")

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

the first one            Barbecue Chicken
the fish                 Smoked Salmon
the chicken              Barbecue Chicken
2nd                      Barbecue Chicken
bbq                      Barbecue Chicken
salmon                   salmon
roasted turkey           Barbecue Chicken
dried halibut            Smoked Salmon


Programmatically print the names of all the model layers and their dimensions.

In [ ]:
for name, param in model.named_parameters():
  print(f"Layer: {name}, Size: {param.size()}")


Layer: shared.weight, Size: torch.Size([32128, 512])
Layer: encoder.block.0.layer.0.SelfAttention.q.weight, Size: torch.Size([384, 512])
Layer: encoder.block.0.layer.0.SelfAttention.k.weight, Size: torch.Size([384, 512])
Layer: encoder.block.0.layer.0.SelfAttention.v.weight, Size: torch.Size([384, 512])
Layer: encoder.block.0.layer.0.SelfAttention.o.weight, Size: torch.Size([512, 384])
Layer: encoder.block.0.layer.0.SelfAttention.relative_attention_bias.weight, Size: torch.Size([32, 6])
Layer: encoder.block.0.layer.0.layer_norm.weight, Size: torch.Size([512])
Layer: encoder.block.0.layer.1.DenseReluDense.wi_0.weight, Size: torch.Size([1024, 512])
Layer: encoder.block.0.layer.1.DenseReluDense.wi_1.weight, Size: torch.Size([1024, 512])
Layer: encoder.block.0.layer.1.DenseReluDense.wo.weight, Size: torch.Size([512, 1024])
Layer: encoder.block.0.layer.1.layer_norm.weight, Size: torch.Size([512])
Layer: encoder.block.1.layer.0.SelfAttention.q.weight, Size: torch.Size([384, 512])
Layer: enco

Programmatically print the total number of parameters/weights in this model.

In [ ]:
sum(p.numel() for p in model.parameters())

76961152

Reset the final layer weights

In [ ]:
# Print the weights of the final layer
if hasattr(model.decoder, 'final_layer_norm'):
    final_layer_weights = model.decoder.final_layer_norm.weight
    print("Weights of the Final Layer:")
    print(final_layer_weights)

Weights of the Final Layer:
Parameter containing:
tensor([ 1.5583e-01,  1.6458e-01,  1.8197e-01,  2.0792e-01,  1.5886e-01,
         1.4222e-01,  1.5845e-01,  1.4269e-01,  1.3648e-01,  1.5702e-01,
         1.6670e-01,  1.3271e-01,  1.7980e-01,  3.2683e-01,  2.0897e-01,
         2.6234e-01,  1.8381e-01,  1.8566e-01,  1.8115e-01,  1.9588e-01,
         1.5456e-01,  2.1353e-01,  1.5126e-01,  1.6348e-01,  1.8062e-01,
         1.4414e-01,  1.7974e-01,  2.0646e-01,  1.7899e-01,  2.0434e-01,
         1.6415e-01,  1.4987e-01,  1.3866e-01,  2.2488e-01,  1.7041e-01,
         6.1698e-01,  1.8228e-01,  1.7578e-01,  1.6113e-01,  2.4024e-01,
         1.6280e-01,  2.2871e-01,  1.6127e-01,  1.8426e-01,  2.1641e-01,
         2.6774e-01,  1.8475e-01,  1.5955e-01,  2.5002e-01,  1.9592e-01,
         1.5467e-01,  2.0025e-01,  1.7020e-01,  1.4393e-01,  1.9788e-01,
         1.5900e-01,  1.4895e-01,  1.5042e-01,  2.6026e-01,  1.5933e-01,
         1.5081e-01,  2.0102e-01,  1.9843e-01,  1.5577e-01,  1.5257e-01,
 

In [ ]:
import torch
with torch.no_grad():
  if hasattr(model.decoder, 'final_layer_norm'):
    model.decoder.final_layer_norm.weight.fill_(0.0)


In [ ]:
# Print the weights of the final layer
if hasattr(model.decoder, 'final_layer_norm'):
    final_layer_weights = model.decoder.final_layer_norm.weight
    print("Weights of the Final Layer:")
    print(final_layer_weights)

Weights of the Final Layer:
Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.

Final layer weights has been reset, now again do the QnA

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")

def query_from_list(query, options):
    t5query = f"""Question: Select the item from this list which is "{query}". Context: * {" * ".join(options)}"""
    inputs = tokenizer(t5query, return_tensors="pt")
    outputs = model.generate(**inputs, max_new_tokens=20)
    return tokenizer.batch_decode(outputs, skip_special_tokens=True)

tests = ["the first one", "the fish", "the chicken", "2nd", "bbq", "salmon", "roasted turkey", "dried halibut"]
options = ["Barbecue Chicken", "Smoked Salmon"]
for t in tests:
    result = query_from_list(t, options)
    print(f"{t:<24} {result[0]}")

the first one            Barbecue Chicken
the fish                 Barbecue Chicken
the chicken              Barbecue Chicken
2nd                      Barbecue Chicken
bbq                      Barbecue Chicken
salmon                   Barbecue Chicken
roasted turkey           Barbecue Chicken
dried halibut            Barbecue Chicken


Custom training

In [ ]:
TOKENIZER = T5TokenizerFast.from_pretrained("google/flan-t5-small")
MODEL = T5ForConditionalGeneration.from_pretrained("google/flan-t5-small", return_dict=True)
OPTIMIZER = Adam(MODEL.parameters(), lr=0.00001)
Q_LEN = 256   # Question Length
T_LEN = 32    # Target Length
BATCH_SIZE = 4

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Download the data set using url. https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json

In [ ]:
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json

--2024-02-24 18:15:13--  https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.109.153, 185.199.111.153, 185.199.108.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.109.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42123633 (40M) [application/json]
Saving to: ‘train-v2.0.json’

train-v2.0.json     100%[===================>]  40.17M   150MB/s    in 0.3s    

2024-02-24 18:15:16 (150 MB/s) - ‘train-v2.0.json’ saved [42123633/42123633]



In [ ]:
with open('train-v2.0.json') as f:
    data = json.load(f)

In [ ]:
# Extracting context, question, and answers from the dataset

def prepare_data(data):
    articles = []

    for article in data["data"]:
        for paragraph in article["paragraphs"]:
            for qa in paragraph["qas"]:
                question = qa["question"]

                if not qa["is_impossible"]:
                  answer = qa["answers"][0]["text"]

                inputs = {"context": paragraph["context"], "question": question, "answer": answer}


                articles.append(inputs)

    return articles

In [ ]:
data = prepare_data(data)

# Create a Dataframe
data = pd.DataFrame(data)

In [ ]:
class QA_Dataset(Dataset):
    def __init__(self, tokenizer, dataframe, q_len, t_len):
        self.tokenizer = tokenizer
        self.q_len = q_len
        self.t_len = t_len
        self.data = dataframe
        self.questions = self.data["question"]
        self.context = self.data["context"]
        self.answer = self.data['answer']

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        question = self.questions[idx]
        context = self.context[idx]
        answer = self.answer[idx]

        question_tokenized = self.tokenizer(question, context, max_length=self.q_len, padding="max_length",
                                                    truncation=True, pad_to_max_length=True, add_special_tokens=True)
        answer_tokenized = self.tokenizer(answer, max_length=self.t_len, padding="max_length",
                                          truncation=True, pad_to_max_length=True, add_special_tokens=True)

        labels = torch.tensor(answer_tokenized["input_ids"], dtype=torch.long)
        labels[labels == 0] = -100

        return {
            "input_ids": torch.tensor(question_tokenized["input_ids"], dtype=torch.long),
            "attention_mask": torch.tensor(question_tokenized["attention_mask"], dtype=torch.long),
            "labels": labels,
            "decoder_attention_mask": torch.tensor(answer_tokenized["attention_mask"], dtype=torch.long)
        }

In [ ]:
# Dataloader

train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

train_sampler = RandomSampler(train_data.index)
val_sampler = RandomSampler(val_data.index)

qa_dataset = QA_Dataset(TOKENIZER, data, Q_LEN, T_LEN)

train_loader = DataLoader(qa_dataset, batch_size=BATCH_SIZE, sampler=train_sampler)
val_loader = DataLoader(qa_dataset, batch_size=BATCH_SIZE, sampler=val_sampler)

In [ ]:
train_loss = 0
val_loss = 0
train_batch_count = 0
val_batch_count = 0

for epoch in range(2):
    MODEL.train()
    for batch in tqdm(train_loader, desc="Training batches"):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]
        decoder_attention_mask = batch["decoder_attention_mask"]

        outputs = MODEL(
                          input_ids=input_ids,
                          attention_mask=attention_mask,
                          labels=labels,
                          decoder_attention_mask=decoder_attention_mask
                        )

        OPTIMIZER.zero_grad()
        outputs.loss.backward()
        OPTIMIZER.step()
        train_loss += outputs.loss.item()
        train_batch_count += 1

    #Evaluation
    MODEL.eval()
    for batch in tqdm(val_loader, desc="Validation batches"):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]
        decoder_attention_mask = batch["decoder_attention_mask"]

        outputs = MODEL(
                          input_ids=input_ids,
                          attention_mask=attention_mask,
                          labels=labels,
                          decoder_attention_mask=decoder_attention_mask
                        )

        OPTIMIZER.zero_grad()
        outputs.loss.backward()
        OPTIMIZER.step()
        val_loss += outputs.loss.item()
        val_batch_count += 1

    print(f"{epoch+1}/{2} -> Train loss: {train_loss / train_batch_count}\tValidation loss: {val_loss/val_batch_count}")

Training batches:   2%|▏         | 440/26064 [35:05<32:01:22,  4.50s/it]

In [ ]:
MODEL.save_pretrained("qa_model")
TOKENIZER.save_pretrained("qa_tokenizer")

# Saved files
"""('qa_tokenizer/tokenizer_config.json',
 'qa_tokenizer/special_tokens_map.json',
 'qa_tokenizer/spiece.model',
'qa_tokenizer/added_tokens.json',
'qa_tokenizer/tokenizer.json')"""

BLEU validation metric

In [ ]:
def predict_answer(context, question, ref_answer=None):
    inputs = TOKENIZER(question, context, max_length=Q_LEN, padding="max_length", truncation=True, add_special_tokens=True)

    input_ids = torch.tensor(inputs["input_ids"], dtype=torch.long).to(DEVICE).unsqueeze(0)
    attention_mask = torch.tensor(inputs["attention_mask"], dtype=torch.long).to(DEVICE).unsqueeze(0)

    outputs = MODEL.generate(input_ids=input_ids, attention_mask=attention_mask)

    predicted_answer = TOKENIZER.decode(outputs.flatten(), skip_special_tokens=True)

    if ref_answer:
        # Load the Bleu metric
        bleu = evaluate.load("google_bleu")
        score = bleu.compute(predictions=[predicted_answer],
                            references=[ref_answer])

        print("Context: \n", context)
        print("\n")
        print("Question: \n", question)
        return {
            "Reference Answer: ": ref_answer,
            "Predicted Answer: ": predicted_answer,
            "BLEU Score: ": score
        }
    else:
        return predicted_answer

In [ ]:
context = data.iloc[0]["context"]
question = data.iloc[0]["question"]
answer = data.iloc[0]["answer"]

predict_answer(context, question, answer)
